# Data Science for Dummies
*A comprehensive guide to the data science pipeline by Elan Naideck and Erika Shlunk*

## Before you begin

## Data Collection

## Data Processing

## Exploratory Data Analysis and Visualization

## Analysis, Hypothesis Testing and ML

## Insight and Policy Decision